In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'e:\\MLs\\MLOPS Project\\SimpleMLopsProject'

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/myst1cop4/SimpleMLopsProject.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'myst1cop4'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '157285d8090e255b7d88529c36dde66cd4937f0f'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    test_data_path: Path
    metric_filename: str
    all_params: dict
    target_column: str
    mlflow_uri: str

In [5]:
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 scherma_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(scherma_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            test_data_path=config.test_data_path,
            metric_filename=config.metric_filename,
            all_params=params,
            target_column=schema.name,
            mlflow_uri=schema.MLFLOW_URI
        )
        
        return model_eval_config

In [8]:
import os
import pandas as pd
import mlflow
import mlflow.sklearn
import joblib
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        r2 = r2_score(actual, pred)
        return rmse, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme     
        
        with mlflow.start_run():
            
            (rmse, r2) = self.eval_metrics(test_y, model.predict(test_x))
            
            scores = {
                "rmse": rmse,
                "r2": r2
            }
            
            save_json(path = Path(self.config.metric_filename), data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name='ElasticNet')
            else:
                mlflow.sklearn.log_model(model, "model") 
    
    